# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 31 2022 11:31AM,238423,10,MET210731,"Methapharm, Inc.",Released
1,May 31 2022 11:31AM,238423,10,MET210732,"Methapharm, Inc.",Released
2,May 31 2022 11:31AM,238423,10,MET210734,"Methapharm, Inc.",Released
3,May 31 2022 11:31AM,238423,10,MET210739,"Methapharm, Inc.",Released
4,May 31 2022 11:31AM,238423,10,MET210740,"Methapharm, Inc.",Released
5,May 31 2022 11:31AM,238423,10,MET210741,"Methapharm, Inc.",Released
6,May 31 2022 11:31AM,238423,10,MET210742,"Methapharm, Inc.",Released
7,May 31 2022 11:31AM,238423,10,MET210743,"Methapharm, Inc.",Released
8,May 31 2022 11:31AM,238423,10,MET210746,"Methapharm, Inc.",Released
9,May 31 2022 11:31AM,238422,10,MET210727,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
58,238417,Released,16
59,238418,Released,11
60,238420,Released,1
61,238422,Released,23
62,238423,Released,9


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238417,NaN,NaN,16.0
238418,NaN,NaN,11.0
238420,NaN,NaN,1.0
238422,NaN,NaN,23.0
238423,NaN,NaN,9.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238290,0.0,10.0,2.0
238316,0.0,2.0,6.0
238317,0.0,0.0,10.0
238319,0.0,0.0,1.0
238334,0.0,3.0,11.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238290,0,10,2
238316,0,2,6
238317,0,0,10
238319,0,0,1
238334,0,3,11


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238290,0,10,2
1,238316,0,2,6
2,238317,0,0,10
3,238319,0,0,1
4,238334,0,3,11


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238290,,10,2
1,238316,,2,6
2,238317,,,10
3,238319,,,1
4,238334,,3,11


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 31 2022 11:31AM,238423,10,"Methapharm, Inc."
9,May 31 2022 11:31AM,238422,10,"Methapharm, Inc."
32,May 31 2022 11:28AM,238420,102,"Senseonics, Incorporated"
33,May 31 2022 11:25AM,238357,20,Reliable 1 Laboratories LLC
34,May 31 2022 11:25AM,238418,20,Alumier Labs Inc.
45,May 31 2022 11:23AM,238417,20,Alumier Labs Inc.
61,May 31 2022 11:20AM,238416,20,Alumier Labs Inc.
79,May 31 2022 11:08AM,238415,10,ISDIN Corporation
110,May 31 2022 11:08AM,238412,10,ISDIN Corporation
155,May 31 2022 11:05AM,238409,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 31 2022 11:31AM,238423,10,"Methapharm, Inc.",,,9
1,May 31 2022 11:31AM,238422,10,"Methapharm, Inc.",,,23
2,May 31 2022 11:28AM,238420,102,"Senseonics, Incorporated",,,1
3,May 31 2022 11:25AM,238357,20,Reliable 1 Laboratories LLC,,,1
4,May 31 2022 11:25AM,238418,20,Alumier Labs Inc.,,,11
5,May 31 2022 11:23AM,238417,20,Alumier Labs Inc.,,,16
6,May 31 2022 11:20AM,238416,20,Alumier Labs Inc.,,,18
7,May 31 2022 11:08AM,238415,10,ISDIN Corporation,,,31
8,May 31 2022 11:08AM,238412,10,ISDIN Corporation,,,45
9,May 31 2022 11:05AM,238409,10,ISDIN Corporation,,,41


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 31 2022 11:31AM,238423,10,"Methapharm, Inc.",9,,
1,May 31 2022 11:31AM,238422,10,"Methapharm, Inc.",23,,
2,May 31 2022 11:28AM,238420,102,"Senseonics, Incorporated",1,,
3,May 31 2022 11:25AM,238357,20,Reliable 1 Laboratories LLC,1,,
4,May 31 2022 11:25AM,238418,20,Alumier Labs Inc.,11,,
5,May 31 2022 11:23AM,238417,20,Alumier Labs Inc.,16,,
6,May 31 2022 11:20AM,238416,20,Alumier Labs Inc.,18,,
7,May 31 2022 11:08AM,238415,10,ISDIN Corporation,31,,
8,May 31 2022 11:08AM,238412,10,ISDIN Corporation,45,,
9,May 31 2022 11:05AM,238409,10,ISDIN Corporation,41,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 31 2022 11:31AM,238423,10,"Methapharm, Inc.",9,,
1,May 31 2022 11:31AM,238422,10,"Methapharm, Inc.",23,,
2,May 31 2022 11:28AM,238420,102,"Senseonics, Incorporated",1,,
3,May 31 2022 11:25AM,238357,20,Reliable 1 Laboratories LLC,1,,
4,May 31 2022 11:25AM,238418,20,Alumier Labs Inc.,11,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 31 2022 11:31AM,238423,10,"Methapharm, Inc.",9.0,NaN,NaN
1,May 31 2022 11:31AM,238422,10,"Methapharm, Inc.",23.0,NaN,NaN
2,May 31 2022 11:28AM,238420,102,"Senseonics, Incorporated",1.0,NaN,NaN
3,May 31 2022 11:25AM,238357,20,Reliable 1 Laboratories LLC,1.0,NaN,NaN
4,May 31 2022 11:25AM,238418,20,Alumier Labs Inc.,11.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 31 2022 11:31AM,238423,10,"Methapharm, Inc.",9.0,0.0,0.0
1,May 31 2022 11:31AM,238422,10,"Methapharm, Inc.",23.0,0.0,0.0
2,May 31 2022 11:28AM,238420,102,"Senseonics, Incorporated",1.0,0.0,0.0
3,May 31 2022 11:25AM,238357,20,Reliable 1 Laboratories LLC,1.0,0.0,0.0
4,May 31 2022 11:25AM,238418,20,Alumier Labs Inc.,11.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,7151756,708.0,44.0,5.0
102,476787,1.0,1.0,0.0
12,1430121,78.0,2.0,0.0
15,953242,19.0,12.0,0.0
18,953545,4.0,0.0,0.0
19,476698,7.0,1.0,4.0
20,1191942,57.0,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,7151756,708.0,44.0,5.0
1,102,476787,1.0,1.0,0.0
2,12,1430121,78.0,2.0,0.0
3,15,953242,19.0,12.0,0.0
4,18,953545,4.0,0.0,0.0
5,19,476698,7.0,1.0,4.0
6,20,1191942,57.0,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,708.0,44.0,5.0
1,102,1.0,1.0,0.0
2,12,78.0,2.0,0.0
3,15,19.0,12.0,0.0
4,18,4.0,0.0,0.0
5,19,7.0,1.0,4.0
6,20,57.0,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,708.0
1,102,Released,1.0
2,12,Released,78.0
3,15,Released,19.0
4,18,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,18,19,20
Status,,,,,,,
Completed,5.0,0.0,0.0,0.0,0.0,4.0,0.0
Executing,44.0,1.0,2.0,12.0,0.0,1.0,3.0
Released,708.0,1.0,78.0,19.0,4.0,7.0,57.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,18,19,20
0,Completed,5.0,0.0,0.0,0.0,0.0,4.0,0.0
1,Executing,44.0,1.0,2.0,12.0,0.0,1.0,3.0
2,Released,708.0,1.0,78.0,19.0,4.0,7.0,57.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,18,19,20
0,Completed,5.0,0.0,0.0,0.0,0.0,4.0,0.0
1,Executing,44.0,1.0,2.0,12.0,0.0,1.0,3.0
2,Released,708.0,1.0,78.0,19.0,4.0,7.0,57.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()